## 1. 조건 & 이동평균

- np.where(조건, 조건이 성립할때 결과, 조건이 아닐 때 결과)
- 이동평균 : DataFrame.rolling(window = 일 수).mean()
- 지수이동평균 : DataFrame.ewm(span = , adjust=False).mean()

In [2]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd

In [3]:
df = fdr.DataReader('006650',start='2021')
df = df[['Close']].copy()
df

,Close
Date,
2021-01-04,234000
2021-01-05,235000
2021-01-06,277000
2021-01-07,290500
2021-01-08,285000
...,...
2025-01-20,90300
2025-01-21,93500
2025-01-22,91800


In [5]:
# 가격변동 -> Change 열 추가
df['Change'] = df['Close'] - df['Close'].shift(1)
df

,Close,Change
Date,,
2021-01-04,234000,NaN
2021-01-05,235000,1000.0
2021-01-06,277000,42000.0
2021-01-07,290500,13500.0
2021-01-08,285000,-5500.0
...,...,...
2025-01-20,90300,6800.0
2025-01-21,93500,3200.0
2025-01-22,91800,-1700.0


In [7]:
df['Gain']=np.where(df['Change'] >= 0, df['Change'],0)
df['Loss']=np.where(df['Change'] < 0, df['Change']*-1, 0)
df

,Close,Change,Gain,Loss
Date,,,,
2021-01-04,234000,NaN,0.0,0.0
2021-01-05,235000,1000.0,1000.0,0.0
2021-01-06,277000,42000.0,42000.0,0.0
2021-01-07,290500,13500.0,13500.0,0.0
2021-01-08,285000,-5500.0,0.0,5500.0
...,...,...,...,...
2025-01-20,90300,6800.0,6800.0,0.0
2025-01-21,93500,3200.0,3200.0,0.0
2025-01-22,91800,-1700.0,0.0,1700.0


In [8]:
## 이동평균

df['avgGain'] = df['Gain'].rolling(window=14).mean()
df['avgLoss'] = df['Loss'].rolling(window=14).mean()
df

,Close,Change,Gain,Loss,avgGain,avgLoss
Date,,,,,,
2021-01-04,234000,NaN,0.0,0.0,NaN,NaN
2021-01-05,235000,1000.0,1000.0,0.0,NaN,NaN
2021-01-06,277000,42000.0,42000.0,0.0,NaN,NaN
2021-01-07,290500,13500.0,13500.0,0.0,NaN,NaN
2021-01-08,285000,-5500.0,0.0,5500.0,NaN,NaN
...,...,...,...,...,...,...
2025-01-20,90300,6800.0,6800.0,0.0,1478.571429,600.000000
2025-01-21,93500,3200.0,3200.0,0.0,1707.142857,578.571429
2025-01-22,91800,-1700.0,0.0,1700.0,1707.142857,592.857143


In [13]:
def rsi_fn(df, rsi_period = 14):

    import numpy as np

    df_close = df[['Close']].copy()
    df_close['Change'] = df_close['Close'] - df_close['Close'].shift(1)
    df_close['Gain'] = np.where(df['Change']>=0,df['Change'],0)
    df_close['Loss'] = np.where(df['Change']<0,df['Change']*-1,0)
    df_close['avgGain'] = df_close['Gain'].rolling(window=rsi_period).mean()
    df_close['avgLoss'] = df_close['Loss'].rolling(window=rsi_period).mean()
    df_close['RS'] = df_close['avgGain'] / df_close['avgLoss']
    df_close['RSI'] = 100 - (100/(1+df_close['RS']))
    return df_close

In [14]:
df_rsi = rsi_fn(df=df,rsi_period=14)
df_rsi.head(20)

,Close,Change,Gain,Loss,avgGain,avgLoss,RS,RSI
Date,,,,,,,,
2021-01-04,234000,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2021-01-05,235000,1000.0,1000.0,0.0,NaN,NaN,NaN,NaN
2021-01-06,277000,42000.0,42000.0,0.0,NaN,NaN,NaN,NaN
2021-01-07,290500,13500.0,13500.0,0.0,NaN,NaN,NaN,NaN
2021-01-08,285000,-5500.0,0.0,5500.0,NaN,NaN,NaN,NaN
2021-01-11,277000,-8000.0,0.0,8000.0,NaN,NaN,NaN,NaN
2021-01-12,282000,5000.0,5000.0,0.0,NaN,NaN,NaN,NaN
2021-01-13,277500,-4500.0,0.0,4500.0,NaN,NaN,NaN,NaN
2021-01-14,284000,6500.0,6500.0,0.0,NaN,NaN,NaN,NaN
